In [1]:
import Data as data

In [2]:
import importlib

In [3]:
importlib.reload(data)

<module 'Data' from 'C:\\Users\\vinayak\\Desktop\\Backtesting\\Data.py'>

In [4]:
import numpy as np
import pandas as pd

In [5]:
import sqlite3

In [6]:
arr = [
    'BANKNIFTY',
    'AXISBANK',
    'BANDHANBNK',
    'KOTAKBANK',
    'HDFCBANK',
    'INDUSINDBK',
    'IDFCFIRSTB',
    'FEDERALBNK',
    'BANKBARODA',
    'AUBANK',
    'ICICIBANK',
    'PNB',
    'SBIN'
]

In [7]:
def get_futures_data_between(symbol, expiry_type, start_date, end_date):
    print(f"Fetching {symbol} Futures from QDAP")
    cursor, conn = data.make_connection_to_db(True)
    query = f'''
        SELECT *
        FROM ohlcv_future_per_minute ofpm
        WHERE ofpm.symbol = '{symbol}'
        AND ofpm.expiry_type = '{expiry_type}'
        AND DATE(ofpm.date_timestamp) >= '{start_date}'
        AND DATE(ofpm.date_timestamp) <= '{end_date}'
        ORDER BY date_timestamp ASC;
    '''
    cursor.execute(query)
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    print(f'{symbol} futures fetched')
    return df

In [8]:
def get_options_data_betweem(symbol, expiry_type, start_date, end_date):
    print(f"Fetching {symbol} Options from QDAP")
    cursor, conn = data.make_connection_to_db(True)
    cursor.execute(
        f'''
            SELECT *
            FROM ohlcv_options_per_minute oopm
            WHERE oopm.symbol = '{symbol}' 
            AND oopm.expiry_type = '{expiry_type}'
            AND DATE(oopm.date_timestamp) >= '{start_date}'
            AND DATE(oopm.date_timestamp) <= '{end_date}'
            ORDER BY date_timestamp ASC;
        '''
    )
    rows = cursor.fetchall()
    cursor.close()
    conn.close()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    print(f'{symbol} options fetched')
    return df

In [9]:
conn = sqlite3.connect('database.db')
conn.close()

In [10]:
def insert_data_into_table(df, table_name):
    conn = sqlite3.connect('database.db')
    df.to_sql(table_name, conn, if_exists='append', index=False)
    conn.close()
    print(f"Data successfully inserted into {table_name} table.")

In [22]:
def f():
    for symbol in arr:
        print(symbol, '\n')
        for start in month_starts:
            print('----------------------------------------------')
            end = start + pd.offsets.MonthEnd(0)
            if end > end_date:
                end = end_date
            print("start, end:", start, end)
            df_futures = get_futures_data_between(symbol, expiry_type, start, end)
            try:
                insert_data_into_table(df_futures, 'ohlcv_future_per_1_minute')
            except Exception as e:
                print(e)
                print(df_futures)
            print('----------------------------------------------\n')
        print("=================================================\n\n")

In [12]:
def g():
    for symbol in arr:
        print(symbol, '\n')
        for start in month_starts:
            print('----------------------------------------------')
            end = start + pd.offsets.MonthEnd(0)
            if end > end_date:
                end = end_date
            print("start, end:", start, end)
            df_options = get_options_data_betweem(symbol, expiry_type, start, end)
            insert_data_into_table(df_options, 'ohlcv_options_per_1_minute')
            print('----------------------------------------------\n')
        print("=================================================\n\n")

In [13]:
expiry_type = 'I'

In [16]:
start_date = '2023-07-01'
end_date = '2024-08-01'

In [17]:
start_date = pd.to_datetime(start_date)
end_date = pd.to_datetime(end_date)

In [18]:
month_starts = pd.date_range(start_date, end_date, freq='MS')

In [19]:
f()

BANKNIFTY 

----------------------------------------------
start, end: 2023-07-01 00:00:00 2023-07-31 00:00:00
Fetching BANKNIFTY Futures from QDAP
BANKNIFTY futures fetched
Data successfully inserted into ohlcv_futures_per_1_minute table.
----------------------------------------------

----------------------------------------------
start, end: 2023-08-01 00:00:00 2023-08-31 00:00:00
Fetching BANKNIFTY Futures from QDAP
BANKNIFTY futures fetched
Data successfully inserted into ohlcv_futures_per_1_minute table.
----------------------------------------------

----------------------------------------------
start, end: 2023-09-01 00:00:00 2023-09-30 00:00:00
Fetching BANKNIFTY Futures from QDAP
BANKNIFTY futures fetched
Data successfully inserted into ohlcv_futures_per_1_minute table.
----------------------------------------------

----------------------------------------------
start, end: 2023-10-01 00:00:00 2023-10-31 00:00:00
Fetching BANKNIFTY Futures from QDAP
BANKNIFTY futures fetche

In [17]:
def query_local(query):
    import sqlite3
    conn = sqlite3.connect('database.db')
    # Example query to fetch all records from the 'options' table
    df = pd.read_sql(query, conn)
    conn.close()
    return df

In [30]:
query = '''
    SELECT *
    FROM bhav_copy_data_future
'''

In [31]:
import pandas as pd

In [32]:
query_local(query)

,symbol,open,high,low,close,open_interest,change_in_open_interest,data_date,id,expiry,expiry_type,contracts_traded,volume
0,BANKNIFTY,4494395,4514215,4462595,4477145,1879020,205155,2023-08-10,40770,2023-08-31,I,249536,3743040
1,BANKNIFTY,4494395,4514215,4462595,4477145,1879020,205155,2023-08-10,179432,2023-08-31,I,249536,3743040
2,BANKNIFTY,4475900,4475900,4439240,4446435,2032845,153825,2023-08-11,41141,2023-08-31,I,187849,2817735
3,BANKNIFTY,4475900,4475900,4439240,4446435,2032845,153825,2023-08-11,179803,2023-08-31,I,187849,2817735
4,BANKNIFTY,4431995,4439895,4398240,4423405,1904655,-128190,2023-08-14,42093,2023-08-31,I,177425,2661375
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2304,AUBANK,117375,118600,115615,116845,4256500,-71000,2021-09-07,499353,2021-09-30,I,3309,1654500
2305,AUBANK,117025,119340,115800,118650,4124500,-132000,2021-09-08,500172,2021-09-30,I,2586,1293000
2306,AUBANK,118005,118925,117300,117685,4126500,2000,2021-09-09,500480,2021-09-30,I,1912,956000
2307,AUBANK,117055,117400,115005,115430,4224500,98000,2021-09-13,500980,2021-09-30,I,2385,1192500


In [33]:
g()

NameError: name 'g' is not defined

In [48]:
import importlib

In [49]:
importlib.reload(data)

<module 'Data' from 'C:\\Users\\vinayak\\Desktop\\Backtesting\\Data.py'>

In [39]:
df1 = data.fetch_futures_data_with_timestamps_between(arr[0], expiry_type, start_date.date(), end_date.date())

Fetching data from Database
BANKNIFTY futures fetched
Saved dataframe to Database\FUTURES\BANKNIFTY_I\TIMESTAMPS_BETWEEN_2023-07-01_2024-08-01.csv


In [44]:
# df2 = data.fetch_options_data_with_expiry_between(arr[0], expiry_type, start_date.date(), end_date.date(), False, 1)

In [42]:
df3 = data.fetch_futures_data_with_timestamps_between(arr[0], expiry_type, start_date.date(), end_date.date(), False, 1)

Fetching data from Database
BANKNIFTY futures fetched
Saved dataframe to Database\FUTURES\BANKNIFTY_I\TIMESTAMPS_BETWEEN_2023-07-01_2024-08-01.csv


In [46]:
def rename_table(old_table_name, new_table_name):
    conn = sqlite3.connect('database.db')
    cursor = conn.cursor()
    cursor.execute(f"ALTER TABLE {old_table_name} RENAME TO {new_table_name};")
    conn.commit()
    conn.close()
    print(f"Table renamed from {old_table_name} to {new_table_name}.")

In [47]:
old_table_name = 'ohlcv_futures_per_1_minute'
new_table_name = 'ohlcv_future_per_1_minute'

In [39]:
cursor, conn = data.make_connection_to_db(False)

In [11]:
conn.execute('CREATE INDEX idx_symbol_date_expiry_ohlcv_future_per_1_minute ON ohlcv_future_per_1_minute (symbol, date_timestamp, expiry_type);')

In [12]:
conn.execute('CREATE INDEX idx_symbol_date_expiry_ohlcv_future_per_15_minute ON ohlcv_future_per_15_minute (symbol, date_timestamp, expiry_type);')

In [13]:
conn.execute('CREATE INDEX idx_symbol_date_expiry_ohlcv_options_per_1_minute ON ohlcv_options_per_1_minute (symbol, date_timestamp, expiry_type);')

In [14]:
conn.execute('CREATE INDEX idx_symbol_date_expiry_ohlcv_options_per_15_minute ON ohlcv_options_per_15_minute (symbol, date_timestamp, expiry_type);')

In [40]:
conn.execute('CREATE INDEX idx_symbol_date_expiry_bhav_copy_data_future ON bhav_copy_data_future (symbol, data_date, expiry_type);')

OperationalError: index idx_symbol_date_expiry_bhav_copy_data_future already exists

In [ ]:
conn.execute('CREATE INDEX idx_symbol_date_expiry_bhav_copy_options_data ON bhav_copy_options_data (symbol, date_timestamp, expiry_type);')

In [41]:
cursor.close()
conn.close()

In [ ]:
rename_table(old_table_name, new_table_name)

In [1]:
import Data as data

In [2]:
from enums import DB

In [ ]:
cursor, conn = data.make_connection_to_db(DB.LocalDB)
create_table_query = '''
    CREATE TABLE IF NOT EXISTS Greeks (
        date_timestamp TEXT,
        symbol TEXT,
        underlying_price REAL,
        option_type TEXT,
        option_strike REAL,
        option_price REAL,
        expiry_date TEXT,
        risk_free_rate REAL,
        dividend_yield REAL,
        implied_volatility REAL,
        delta REAL,
        gamma REAL,
        vega REAL,
        theta REAL,
        rho REAL
    );
'''
cursor.execute(create_table_query)
conn.commit()
cursor.close()
conn.close()


In [4]:
data.query_db(DB.LocalDB, "CREATE INDEX idx_greeks_composite_extended ON Greeks (date_timestamp, symbol, option_type, option_strike, expiry_date, underlying_price);")

In [22]:
query_create_greeks_2 = '''
    CREATE TABLE IF NOT EXISTS Greeks_2 (
        underlying_price REAL,
        option_strike REAL,
        option_price REAL,
        time_to_expiry REAL,
        option_type TEXT,
        implied_volatility REAL,
        delta REAL,
        gamma REAL,
        vega REAL,
        theta REAL,
        rho REAL
    );
'''
query_create_index_greeks_2 = '''
    CREATE INDEX 
    idx_greeks_composite_extended_greeks_2
    ON Greeks_2 (
    underlying_price,
    option_strike, 
    time_to_expiry, 
    option_type
    );
    '''

In [8]:
from Modules import DataManual as data

In [9]:
from Modules.enums import DB

In [20]:
data.query_db(DB.LocalDB, query_create_greeks_2)

In [23]:
data.query_db(DB.LocalDB, query_create_index_greeks_2)

In [147]:
query1 = 'CREATE INDEX idx_symbol_date_expiry_ohlcv_futures_per_1_minute ON ohlcv_futures_per_1_minute (symbol, date, expiry_type);'
query2 = 'CREATE INDEX idx_symbol_date_expiry_ohlcv_futurse_per_15_minute ON ohlcv_futures_per_15_minute (symbol, date, expiry_type);'
query3 = 'CREATE INDEX idx_symbol_date_expiry_ohlcv_options_per_1_minute ON ohlcv_options_per_1_minute (symbol, date, expiry_type);'
query4 = 'CREATE INDEX idx_symbol_date_expiry_ohlcv_options_per_15_minute ON ohlcv_options_per_15_minute (symbol, date, expiry_type);'
query5 = 'CREATE INDEX idx_symbol_date_expiry_bhav_copy_data_future ON bhav_copy_data_future (symbol, data_date, expiry_type);'
query6 = 'CREATE INDEX idx_symbol_date_expiry_bhav_copy_options_data ON bhav_copy_options_data (symbol, data_date, expiry_type);'

In [155]:
data.query_db(DB.LocalDB, query5)

In [55]:
from Modules import DataManual as data

In [56]:
from Modules import Data_Processing as dp

In [57]:
from Modules.enums import FNO, DB
import pandas as pd

In [58]:
import importlib

In [10]:
data.query_db(DB.LocalDB, query1)

Error Occured in Querying | no such table: main.ohlcv_future_per_1_minute


In [123]:
importlib.reload(data)

<module 'Modules.DataManual' from 'c:\\Users\\vinayak\\Desktop\\Backtesting\\Modules\\DataManual.py'>

In [124]:
df_options_1 = data.fetch_options_data_on('BANKNIFTY', 'I', '2023-08-08', DB.QDAP)

In [125]:
df_options_1['date'] = pd.to_datetime(df_options_1['date_timestamp']).dt.date

In [126]:
df_options_1

,symbol,date_timestamp,open,high,low,close,volume,opt_type,strike,expiry_type,id,expiry,date
0,BANKNIFTY,2023-08-08 09:15:00,362.15,362.15,357.70,360.45,210,PE,44600.0,I,63041221,2023-08-31,2023-08-08
1,BANKNIFTY,2023-08-08 09:15:00,107.85,113.20,107.85,109.85,420,CE,46300.0,I,63050998,2023-08-31,2023-08-08
2,BANKNIFTY,2023-08-08 09:15:00,14.55,14.55,14.55,14.55,15,PE,39000.0,I,62969336,2023-08-31,2023-08-08
3,BANKNIFTY,2023-08-08 09:15:00,804.95,836.80,804.95,825.65,4005,CE,44500.0,I,63040186,2023-08-31,2023-08-08
4,BANKNIFTY,2023-08-08 09:15:00,463.70,487.85,463.70,477.05,945,CE,45100.0,I,63044496,2023-08-31,2023-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25403,BANKNIFTY,2023-08-08 15:29:00,8.25,8.50,8.25,8.50,15,PE,37500.0,I,62969309,2023-08-31,2023-08-08
25404,BANKNIFTY,2023-08-08 15:29:00,32.80,32.85,32.80,32.85,30,PE,42200.0,I,63033586,2023-08-31,2023-08-08
25405,BANKNIFTY,2023-08-08 15:29:00,363.00,365.20,361.45,365.20,120,PE,44700.0,I,63042269,2023-08-31,2023-08-08
25406,BANKNIFTY,2023-08-08 15:29:00,44.50,44.65,44.40,44.65,45,PE,42600.0,I,63034172,2023-08-31,2023-08-08


In [127]:
data.insert_data_into_local_database(df_options_1, 'ohlcv_options_per_1_minute')

Data successfully inserted into Local DB: ohlcv_options_per_1_minute table.


In [128]:
df_options_15 = dp.resample_df_to_timeframe(df_options_1, 15, FNO.OPTIONS)

Successfully resampled data to 15mins timeframe


In [129]:
df_options_15['date'] = pd.to_datetime(df_options_15['date_timestamp']).dt.date

In [130]:
df_options_15

,opt_type,strike,date_timestamp,open,high,low,close,volume,expiry,expiry_type,symbol,date
2279,PE,44100.0,2023-08-08 09:15:00,206.85,217.65,202.70,209.95,855,2023-08-31,I,BANKNIFTY,2023-08-08
1416,CE,50500.0,2023-08-08 09:15:00,9.80,9.90,9.65,9.90,345,2023-08-31,I,BANKNIFTY,2023-08-08
2604,PE,45400.0,2023-08-08 09:15:00,750.05,759.20,729.75,759.20,2115,2023-08-31,I,BANKNIFTY,2023-08-08
1391,CE,50000.0,2023-08-08 09:15:00,10.55,11.05,10.55,11.00,1230,2023-08-31,I,BANKNIFTY,2023-08-08
1366,CE,49500.0,2023-08-08 09:15:00,11.45,14.45,11.45,14.10,1080,2023-08-31,I,BANKNIFTY,2023-08-08
...,...,...,...,...,...,...,...,...,...,...,...,...
315,CE,44000.0,2023-08-08 15:15:00,1204.15,1215.35,1204.15,1213.00,3750,2023-08-31,I,BANKNIFTY,2023-08-08
1535,PE,37500.0,2023-08-08 15:15:00,8.25,9.50,8.25,8.50,405,2023-08-31,I,BANKNIFTY,2023-08-08
1978,PE,42800.0,2023-08-08 15:15:00,52.40,52.65,51.75,52.05,3915,2023-08-31,I,BANKNIFTY,2023-08-08
1415,CE,50000.0,2023-08-08 15:15:00,10.80,10.90,10.25,10.35,2580,2023-08-31,I,BANKNIFTY,2023-08-08


In [131]:
data.insert_data_into_local_database(df_options_15, 'ohlcv_options_per_15_minute')

Data successfully inserted into Local DB: ohlcv_options_per_15_minute table.


In [132]:
df_futures_1 = data.fetch_futures_data_on('BANKNIFTY', 'I', '2023-08-11', DB.QDAP)

In [133]:
df_futures_1['date'] = pd.to_datetime(df_futures_1['date_timestamp']).dt.date

In [134]:
data.insert_data_into_local_database(df_futures_1, 'ohlcv_futures_per_1_minute')

Data successfully inserted into Local DB: ohlcv_futures_per_1_minute table.


In [135]:
df_futures_15 = dp.resample_df_to_timeframe(df_futures_1, 15, FNO.FUTURES)

Successfully resampled data to 15mins timeframe


In [136]:
df_futures_15['date'] = pd.to_datetime(df_futures_15['date_timestamp']).dt.date

In [137]:
data.insert_data_into_local_database(df_futures_15, 'ohlcv_futures_per_15_minute')

Data successfully inserted into Local DB: ohlcv_futures_per_15_minute table.


In [154]:
data.get_bhav_data_futures_on('BANKNIFTY', 'I', '2023-08-11')

Checking if BHAV Futures Data exists for this day in Local DB
Data for this day does not exist in Local DB
Checking if data exists for this day in QDAP
Data Found for this day in QDAP
Data successfully inserted into Local DB: bhav_copy_data_future table.
-----------------------------------------------------------------


,symbol,open,high,low,close,open_interest,change_in_open_interest,date_timestamp,id,expiry,expiry_type,contracts_traded,volume
0,BANKNIFTY,44759.0,44759.0,44392.4,44464.35,2032845,153825,2023-08-11,41141,2023-08-31,I,187849,2817735
1,BANKNIFTY,44759.0,44759.0,44392.4,44464.35,2032845,153825,2023-08-11,179803,2023-08-31,I,187849,2817735


In [138]:
import pandas as pd

In [142]:
data.get_options_data_on('BANKNIFTY', 'I', '2023-08-21', 15)

Checking if Options Data exists for this day in Local DB for timeframe 15mins
Local DB does not contain this day's data for timeframe 15mins
Checking if data exists for this day in Local DB for timeframe 1mins
Local DB does not contain this day's data for timeframe 1mins
Checking if data exists for this day in QDAP for timeframe 1mins
Data Found for this day in QDAP for timeframe 1mins
Data successfully inserted into Local DB: ohlcv_options_per_1_minute table.
Successfully resampled data to 15mins timeframe
Data successfully inserted into Local DB: ohlcv_options_per_15_minute table.
-----------------------------------------------------------------


,opt_type,strike,date_timestamp,open,high,low,close,volume,expiry,expiry_type,symbol,date
1805,CE,50500.0,2023-08-21 09:15:00,7.10,7.10,6.45,6.45,285,2023-08-31,I,BANKNIFTY,2023-08-21
2430,PE,42500.0,2023-08-21 09:15:00,52.00,52.00,36.00,36.40,48105,2023-08-31,I,BANKNIFTY,2023-08-21
176,CE,42500.0,2023-08-21 09:15:00,1549.00,1590.00,1541.70,1574.35,1950,2023-08-31,I,BANKNIFTY,2023-08-21
1164,CE,46500.0,2023-08-21 09:15:00,12.60,13.00,11.15,12.00,33735,2023-08-31,I,BANKNIFTY,2023-08-21
2455,PE,42600.0,2023-08-21 09:15:00,54.80,54.80,40.70,40.70,12840,2023-08-31,I,BANKNIFTY,2023-08-21
...,...,...,...,...,...,...,...,...,...,...,...,...
2354,PE,42000.0,2023-08-21 15:15:00,18.15,18.95,17.65,17.85,54300,2023-08-31,I,BANKNIFTY,2023-08-21
538,CE,43900.0,2023-08-21 15:15:00,451.40,455.50,436.95,444.75,46260,2023-08-31,I,BANKNIFTY,2023-08-21
1263,CE,46800.0,2023-08-21 15:15:00,8.30,8.30,7.90,7.90,1725,2023-08-31,I,BANKNIFTY,2023-08-21
2379,PE,42100.0,2023-08-21 15:15:00,19.70,20.65,15.35,19.85,4620,2023-08-31,I,BANKNIFTY,2023-08-21
